# bert model

In [20]:
import torch
import time
import inspect
import math
import numpy as np
import torch.nn as nn
from torchinfo import summary

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
model_bert = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')

Using cache found in C:\Users\vd00r/.cache\torch\hub\huggingface_pytorch-transformers_main
c:\Users\vd00r\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
codes_index = ["N","E-2", "E-3", "E-4", "E-5", "E-6", "E-7", "E-8", "C-1", "C-2", "C-3", "C-4", "I-1", "I-2", "I-3", "I-4", "I-5", "I-6", "Ro-1:",             "Ro-7", "Ro-9", "Ro-10", "D-1", "D-2", "D-3", "D-4", "D-5", "D-6", "D-7", "D-8", "D-9", "D-10"]

In [23]:
class config:
    n_embd = 768
    n_classes = 32

In [202]:
class bert_classifier(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config
        self.model_bert = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')
        self.lin = nn.Linear(config.n_embd,config.n_classes)
        #self.softmax = nn.softmax()
    def forward(self,x):
        x = self.model_bert(x)
        x = self.lin(x[1])
        return x

In [203]:
model = bert_classifier(config).to(device)

Using cache found in C:\Users\vd00r/.cache\torch\hub\huggingface_pytorch-transformers_main


In [204]:
for param in model.named_parameters():
    print(param[0])

model_bert.embeddings.word_embeddings.weight
model_bert.embeddings.position_embeddings.weight
model_bert.embeddings.token_type_embeddings.weight
model_bert.embeddings.LayerNorm.weight
model_bert.embeddings.LayerNorm.bias
model_bert.encoder.layer.0.attention.self.query.weight
model_bert.encoder.layer.0.attention.self.query.bias
model_bert.encoder.layer.0.attention.self.key.weight
model_bert.encoder.layer.0.attention.self.key.bias
model_bert.encoder.layer.0.attention.self.value.weight
model_bert.encoder.layer.0.attention.self.value.bias
model_bert.encoder.layer.0.attention.output.dense.weight
model_bert.encoder.layer.0.attention.output.dense.bias
model_bert.encoder.layer.0.attention.output.LayerNorm.weight
model_bert.encoder.layer.0.attention.output.LayerNorm.bias
model_bert.encoder.layer.0.intermediate.dense.weight
model_bert.encoder.layer.0.intermediate.dense.bias
model_bert.encoder.layer.0.output.dense.weight
model_bert.encoder.layer.0.output.dense.bias
model_bert.encoder.layer.0.outp

In [279]:
#for name, layer in model_bert.encoder.named_children():
        #print(layer)
layer_names = []
for param in model.named_parameters():
     layer_names.append(param[0])
layer_names.reverse()
print(layer_names)

['lin.bias', 'lin.weight', 'model_bert.pooler.dense.bias', 'model_bert.pooler.dense.weight', 'model_bert.encoder.layer.11.output.LayerNorm.bias', 'model_bert.encoder.layer.11.output.LayerNorm.weight', 'model_bert.encoder.layer.11.output.dense.bias', 'model_bert.encoder.layer.11.output.dense.weight', 'model_bert.encoder.layer.11.intermediate.dense.bias', 'model_bert.encoder.layer.11.intermediate.dense.weight', 'model_bert.encoder.layer.11.attention.output.LayerNorm.bias', 'model_bert.encoder.layer.11.attention.output.LayerNorm.weight', 'model_bert.encoder.layer.11.attention.output.dense.bias', 'model_bert.encoder.layer.11.attention.output.dense.weight', 'model_bert.encoder.layer.11.attention.self.value.bias', 'model_bert.encoder.layer.11.attention.self.value.weight', 'model_bert.encoder.layer.11.attention.self.key.bias', 'model_bert.encoder.layer.11.attention.self.key.weight', 'model_bert.encoder.layer.11.attention.self.query.bias', 'model_bert.encoder.layer.11.attention.self.query.weig

In [260]:
summary(model,input_data=torch.ones([1,512],dtype=torch.int32).to(device))
#print(model(torch.ones([1,512],dtype=torch.int32)).shape)

Layer (type:depth-idx)                                       Output Shape              Param #
bert_classifier                                              [1, 32]                   --
├─BertModel: 1-1                                             [1, 768]                  --
│    └─BertEmbeddings: 2-1                                   [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             23,440,896
│    │    └─Embedding: 3-2                                   [1, 512, 768]             1,536
│    │    └─Embedding: 3-3                                   [1, 512, 768]             393,216
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─BertEncoder: 2-2                                      [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --             

In [261]:
from transformers import BertForSequenceClassification, BertTokenizer

In [264]:
tokenizer = BertTokenizer(#vocab_file)
input = "They shot at him, but they aimed at all of us. The assasination attempt on President Donald J. Trump yesterday was not just an isolated event. It was the culmination of years of attacks being leveled at the President, and all Republicans. We are in a battle for the soul of our Nation. This is not simply about who will be President, but what course our country will take for generations to come. They’ve indicted him, attacked him, impeached him and shot him. He’s still standing. And we need to stand with him. Every election this year is important. In August, and November, we need leaders in office who will FIGHT for our state and nation. I continue to pray for the families of those who lost loved ones yesterday, and for the Trump family, but I’m ready to get out and fight for our country and vote for our future in the coming elections. Will you join me?"
tok = tokenizer.encode_plus(input)
print(len(tok["input_ids"]))

SyntaxError: '(' was never closed (2267807764.py, line 1)

# training loop

In [265]:
#-----TRAINING-HYPERPARAMETERS-----#
max_lr = 1#3e-4
min_lr = max_lr * 0.01
warmup_steps = 100
max_steps = 3000
theta = 0.9
#-----TRAINING-HYPERPARAMETERS-----#

In [ ]:
#-----BATCH-SIZE/GRADIENT-ACCUMILATION-----#
batch_size = 1024
B = 4
grad_accum_steps = batch_size // B
loader = dataloader(B = B)
val_loader = dataloader(B = B, split="val")
print(grad_accum_steps)
#-----BATCH-SIZE/GRADIENT-ACCUMILATION-----#

In [266]:
#-----LEARNING-RATE-SCHEDULER-----#
def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay = it-warmup_steps
    return max_lr - decay * (max_lr/(max_steps-warmup_steps))
#-----LEARNING-RATE-SCHEDULER-----#

In [292]:
for param in model.parameters():
     param.requires_grad = False
for param in model.model_bert.pooler.parameters():
     param.requires_grad = True
for param in model.lin.parameters():
     param.requires_grad = True

In [293]:
#-----GRADUAL-FREEZING----#
class gradual_freezing():
    def __init__(self):
        self.unlock_ratio = 0.083
        self.i = 12
    def check_unfreeze(self,it):
        ratio = it/max_steps
        #print(ratio)
        #print(self.unlock_ratio)
        if ratio > self.unlock_ratio and self.i != 0:
            self.unlock_ratio = self.unlock_ratio +  0.083
            self.i = self.i - 1
            print(self.i)
            for param in model.model_bert.encoder.layer[self.i].parameters():
                param.requires_grad = True
                #print(param[0])
            return True
        return False
#-----GRADUAL-FREEZING-----#
#a = gradual_freezing()
#for g in range(2000):
    #a.check_unfreeze(g)

In [294]:
#-----DISCRIMINATIVE-FINE-TUNING-----#
def disc_fine_tuning(lr):
    parameters = []
    list1 = []
    previous = layer_names[2].split('.')[2]
    lr_disc = lr
    for idx,name in enumerate(layer_names):
        if name.split(".")[0] == "lin":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr})
        if name.split(".")[1] == "pooler":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr_disc})
                    list1.append(p[0])
        if name.split(".")[1] == "embeddings":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr_disc})
        if name.split(".")[1] == "encoder":
                current = name.split('.')[3]
                if current != previous:
                    lr_disc = theta*lr_disc
                previous = current
                for p in model.named_parameters():
                    if p[0] == name and p[1].requires_grad:
                        parameters.append({ "params":p[1] ,"lr":lr_disc})
                        list1.append(p[0])
    return parameters, list1
#-----DISCRIMINATIVE-FINE-TUNING-----#
parameter,list1 = disc_fine_tuning(1)
print(parameter)
optimizer = torch.optim.Adam(parameter)


[{'params': Parameter containing:
tensor([ 0.0124, -0.0055,  0.0181,  0.0077,  0.0319,  0.0086,  0.0003, -0.0271,
         0.0104, -0.0144,  0.0065,  0.0318,  0.0106,  0.0336,  0.0097, -0.0029,
         0.0127, -0.0134,  0.0276, -0.0213, -0.0202, -0.0026,  0.0145,  0.0283,
         0.0065,  0.0337, -0.0233,  0.0121,  0.0277,  0.0320, -0.0238, -0.0233],
       device='cuda:0', requires_grad=True), 'lr': 1}, {'params': Parameter containing:
tensor([[-0.0314, -0.0241,  0.0150,  ...,  0.0191, -0.0291,  0.0134],
        [-0.0059,  0.0165, -0.0024,  ..., -0.0281, -0.0257, -0.0268],
        [ 0.0043,  0.0048,  0.0176,  ..., -0.0028,  0.0354, -0.0021],
        ...,
        [ 0.0010, -0.0286, -0.0064,  ...,  0.0352,  0.0002,  0.0291],
        [ 0.0051,  0.0347,  0.0124,  ...,  0.0214, -0.0196,  0.0236],
        [ 0.0038, -0.0122, -0.0261,  ...,  0.0347,  0.0233,  0.0263]],
       device='cuda:0', requires_grad=True), 'lr': 1}, {'params': Parameter containing:
tensor([-3.5976e-02, -3.8954e-03,  

In [291]:
for param_group in optimizer.param_groups:
        param_group = parameter

In [ ]:
#-----MISC-----#
checkpoint = 0
torch.set_float32_matmul_precision("high")
#-----MISC-----#

In [ ]:
for step in range(max_steps):
    last_step = (step == max_steps - 1)
    t0 = time.time()
    #optimizer.zero_grad(set_to_none=True)
    
    #validation loss
    if step % 250 == 1 or last_step:
        model.eval()
        val_loader.reset()
        with torch.no_grad():
            correct_acum = 0
            val_accum = 0
            val_loss_steps = 80
            for val_steps in range(val_loss_steps):
                x, y = val_loader.next_batch()
                x,y = x.to(device), y.to(device)
                with torch.autocast(device_type=device, dtype=torch.bfloat16):
                    logits,loss = model(x,y)
                values,ind = torch.max(logits,dim = 1)
                correct = np.sum((torch.eq(ind.to("cpu"),y.to("cpu")).numpy()))
                correct_acum = correct_acum + correct
                val_accum = val_accum + loss.detach()
            accuracy = correct_acum / (val_loss_steps * B)
            val_loss = val_accum / val_loss_steps
    
    #save checkpoint
    if step % 500 == 0:
        torch.save(model.state_dict(),f"checkpoint{checkpoint}.pt")
        checkpoint = checkpoint+1

    #minibatch process
    loss_accum = 0
    model.train()
    for mini_step in range(grad_accum_steps):
        x, y = loader.next_batch()
        x,y = x.to(device), y.to(device)
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits,loss = model(x,y)
        #print(f"loss mini_step = {loss} | ministep = {mini_step}")
        loss = loss / grad_accum_steps
        loss_accum = loss_accum + loss.detach()
        loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 7.0)

    t1 = time.time()
    dt = t1-t0

    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group = parameter
    optimizer.step()

torch.save(model.state_dict(),"Final.pt")

# misc

In [ ]:
#-----OPTIMIZER-----#
def configure_optimizers(model, weight_decay, learning_rate, device):
    param_dict = {pn: p for pn, p in model.named_parameters()}
    param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
    decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
    nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
    optim_groups = [
        {'params': decay_params, 'weight_decay': weight_decay},
        {'params': nodecay_params, 'weight_decay': 0.0}
    ]
    num_decay_params = sum(p.numel() for p in decay_params)
    num_nodecay_params = sum(p.numel() for p in nodecay_params) 
    print(num_decay_params,num_nodecay_params)
    fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
    use_fused = fused_available and device == "cuda"
    print(f"using fused AdamW: {use_fused}")
    optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.999), eps=1e-8, fused=use_fused)
    return optimizer  
  
optimizer = configure_optimizers(model = model, weight_decay=0.05, learning_rate=6e-4, device=device)
#-----OPTIMIZER-----#